poverty: by town <br>
hs_completion: by town <br>
n_households: by zip <br>
med_income: by zip <br>
population & density: by zip <br>
crime_rate: by town <br>
students_per_teacher: by town

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, r2_score
from sklearn.decomposition import PCA

import requests
from bs4 import BeautifulSoup

import pickle
import time

In [2]:
inc = pd.read_csv('../data/inc.csv')

In [3]:
pd.set_option('display.max_columns', 30)
inc.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,state,town,poverty,hs_completion,townstate,ZCTA,n_households,med_income,population,density,city,state_id,state_name,lat,lng,crime_rate,students_per_teacher,property_crime,violent_crime
0,0,0,0,Alabama,Abbeville,20.7,79.1,"Abbeville, Alabama",36310,2555,40186,6055,12.2,Abbeville,AL,Alabama,31.60296,-85.21630,62.0,13.000000,51.0,11.0
1,1,1,1,Alabama,Adamsville,16,83.5,"Adamsville, Alabama",35005,2792,50271,7528,89.3,Adamsville,AL,Alabama,33.59515,-87.00089,269.0,17.666667,250.0,19.0
2,2,2,2,Alabama,Addison,34.2,85.1,"Addison, Alabama",35540,1075,45074,2436,12.6,Addison,AL,Alabama,34.23388,-87.18817,15.0,15.500000,14.0,1.0
3,3,3,3,Alabama,Akron,48.7,62.8,"Akron, Alabama",35441,372,36731,1089,6.1,Akron,AL,Alabama,32.85374,-87.73861,NaN,NaN,NaN,NaN
4,4,4,4,Alabama,Alabaster,11.1,88.8,"Alabaster, Alabama",35007,8628,69691,26328,270.8,Alabaster,AL,Alabama,33.21591,-86.79717,585.0,18.571429,488.0,97.0


### Get home price

In [4]:
homes = pd.read_csv('../data/home_values.csv', encoding = 'ISO-8859-1')

In [5]:
homes.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,...,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01
0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,229111.0,229636.0,230273.0,230160.0,230967.0,231319.0,232924.0,234158.0,...,1303420,1295354,1306300,1306459,1316079,1303286,1299284,1286935,1276551,1256582,1241230,1228147,1223980,1219772,1199640
1,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,205061.0,204510.0,204689.0,204481.0,205075.0,205944.0,207421.0,209047.0,...,488358,488781,486322,483680,482143,482338,482361,481827,482027,481365,480344,479200,478010,476998,476589
2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,240264.0,241504.0,242520.0,242857.0,242958.0,243340.0,244063.0,245033.0,...,1245536,1214867,1185142,1163868,1160965,1153839,1150106,1144748,1142054,1135119,1121649,1114776,1120736,1143854,1168729
3,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,4,199134.0,197997.0,197332.0,198094.0,197782.0,197129.0,196340.0,197241.0,...,336222,335882,336060,336006,336292,335970,335716,335633,336003,336046,336119,336002,335757,335553,334918
4,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,5,276740.0,276073.0,276160.0,275674.0,276226.0,277289.0,279198.0,281439.0,...,637028,637817,634400,630912,628667,628999,629012,628193,628163,627519,626348,625467,624255,623172,623088


In [6]:
homes = homes[['RegionName', '2018-08']]
homes.columns = ['zip', 'home_price']

In [7]:
homes.head()

,zip,home_price
0,10025,1293069
1,60657,485709
2,10023,1222255
3,77494,335430
4,60614,633437


In [8]:
inc = pd.merge(inc, homes, left_on = 'ZCTA', right_on = 'zip')

In [9]:
inc.loc[inc['ZCTA'] != inc['zip']]

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,state,town,poverty,hs_completion,townstate,ZCTA,n_households,med_income,population,density,city,state_id,state_name,lat,lng,crime_rate,students_per_teacher,property_crime,violent_crime,zip,home_price


### Get counties for gini index

In [10]:
counties = pd.read_csv('../data/uscities.csv')

In [11]:
counties.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,county_fips_all,county_name_all,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,South Creek,South Creek,WA,Washington,53053,Pierce,53053,Pierce,46.9994,-122.3921,2500.0,125.0,polygon,False,True,America/Los_Angeles,3,98580 98387 98338,1840042075
1,Roslyn,Roslyn,WA,Washington,53037,Kittitas,53037,Kittitas,47.2507,-121.0989,947.0,84.0,polygon,False,True,America/Los_Angeles,3,98941 98068 98925,1840019842
2,Sprague,Sprague,WA,Washington,53043,Lincoln,53043,Lincoln,47.3048,-117.9713,441.0,163.0,polygon,False,True,America/Los_Angeles,3,99032,1840021107
3,Gig Harbor,Gig Harbor,WA,Washington,53053,Pierce,53053,Pierce,47.3352,-122.5968,9507.0,622.0,polygon,False,True,America/Los_Angeles,3,98332 98335,1840019855
4,Lake Cassidy,Lake Cassidy,WA,Washington,53061,Snohomish,53061,Snohomish,48.0639,-122.0920,3591.0,131.0,polygon,False,True,America/Los_Angeles,3,98223 98258 98270,1840041959


In [12]:
counties['townstate'] = counties['city'] + ', ' + counties['state_name']

In [13]:
tempcounties = counties[['county_name', 'townstate']]

In [14]:
print(inc.shape)
inc = pd.merge(inc, tempcounties, left_on = 'townstate', right_on = 'townstate')
print(inc.shape)

(19246, 24)
(18791, 25)


In [15]:
inc['countystate'] = inc['county_name'] + ', ' + inc['state']

### Get Gini

In [16]:
gini = pd.read_csv('../data/gini.csv')

In [17]:
gini.head()

,GEO_ID,NAME,B19083_001E,B19083_001M
0,id,Geographic Area Name,Estimate!!Gini Index,Margin of Error!!Gini Index
1,0500000US01003,"Baldwin County, Alabama",0.4725,0.0271
2,0500000US01015,"Calhoun County, Alabama",0.4650,0.0292
3,0500000US01043,"Cullman County, Alabama",0.4981,0.0572
4,0500000US01049,"DeKalb County, Alabama",0.5003,0.0416


In [18]:
gini.shape

(839, 4)

In [19]:
gini = gini[['NAME', 'B19083_001E']]

In [20]:
gini.drop([0], inplace = True)
gini.reset_index(drop = True, inplace = True)

In [21]:
gini.columns = ['countystate', 'gini']

In [22]:
gini.head()

,countystate,gini
0,"Baldwin County, Alabama",0.4725
1,"Calhoun County, Alabama",0.4650
2,"Cullman County, Alabama",0.4981
3,"DeKalb County, Alabama",0.5003
4,"Elmore County, Alabama",0.4162


In [23]:
def clean_county(county):
    words = [' County', ' Parish', ' Municipio', ' city']
    for word in words:
        if word in county:
            return county.replace(word, '')
    return county

In [24]:
gini['countystate'] = gini['countystate'].apply(clean_county)

In [25]:
gini[gini['countystate'].duplicated(keep=False)]

,countystate,gini
302,"Baltimore, Maryland",0.4499
315,"Baltimore, Maryland",0.5158
396,"St. Louis, Missouri",0.4920
397,"St. Louis, Missouri",0.5056
759,"Roanoke, Virginia",0.4088
772,"Roanoke, Virginia",0.5099


In [26]:
# todo: average among duplicates (few duplicates)
gini = gini.drop_duplicates(subset='countystate', keep='first').reset_index(drop=True)

In [27]:
gini.head()

,countystate,gini
0,"Baldwin, Alabama",0.4725
1,"Calhoun, Alabama",0.4650
2,"Cullman, Alabama",0.4981
3,"DeKalb, Alabama",0.5003
4,"Elmore, Alabama",0.4162


### Unemployment

In [28]:
unemp = pd.read_csv('../data/unemp_rates.csv')

In [29]:
unemp.head()

,GEO_ID,NAME,S2301_C01_001E,S2301_C01_001M,S2301_C02_001E,S2301_C02_001M,S2301_C03_001E,S2301_C03_001M,S2301_C04_001E,S2301_C04_001M,S2301_C01_002E,S2301_C01_002M,S2301_C02_002E,S2301_C02_002M,S2301_C03_002E,...,S2301_C01_034M,S2301_C02_034E,S2301_C02_034M,S2301_C03_034E,S2301_C03_034M,S2301_C04_034E,S2301_C04_034M,S2301_C01_035E,S2301_C01_035M,S2301_C02_035E,S2301_C02_035M,S2301_C03_035E,S2301_C03_035M,S2301_C04_035E,S2301_C04_035M
0,id,Geographic Area Name,Estimate!!Total!!Population 16 years and over,Margin of Error!!Total MOE!!Population 16 year...,Estimate!!Labor Force Participation Rate!!Popu...,Margin of Error!!Labor Force Participation Rat...,Estimate!!Employment/Population Ratio!!Populat...,Margin of Error!!Employment/Population Ratio M...,Estimate!!Unemployment rate!!Population 16 yea...,Margin of Error!!Unemployment rate MOE!!Popula...,Estimate!!Total!!Population 16 years and over!...,Margin of Error!!Total MOE!!Population 16 year...,Estimate!!Labor Force Participation Rate!!Popu...,Margin of Error!!Labor Force Participation Rat...,Estimate!!Employment/Population Ratio!!Populat...,...,Margin of Error!!Total MOE!!EDUCATIONAL ATTAIN...,Estimate!!Labor Force Participation Rate!!EDUC...,Margin of Error!!Labor Force Participation Rat...,Estimate!!Employment/Population Ratio!!EDUCATI...,Margin of Error!!Employment/Population Ratio M...,Estimate!!Unemployment rate!!EDUCATIONAL ATTAI...,Margin of Error!!Unemployment rate MOE!!EDUCAT...,Estimate!!Total!!EDUCATIONAL ATTAINMENT!!Popul...,Margin of Error!!Total MOE!!EDUCATIONAL ATTAIN...,Estimate!!Labor Force Participation Rate!!EDUC...,Margin of Error!!Labor Force Participation Rat...,Estimate!!Employment/Population Ratio!!EDUCATI...,Margin of Error!!Employment/Population Ratio M...,Estimate!!Unemployment rate!!EDUCATIONAL ATTAI...,Margin of Error!!Unemployment rate MOE!!EDUCAT...
1,0500000US01117,"Shelby County, Alabama",173097,1697,64.1,2.1,62.0,2.0,3.2,0.9,11580,1002,38.5,7.8,34.5,...,3233,80.4,3.6,78.6,3.7,1.9,1.3,49409,2772,85.2,2.5,83.8,2.8,1.5,0.9
2,0500000US01121,"Talladega County, Alabama",65419,392,53.5,3.2,48.8,3.0,8.5,2.4,4902,793,53.8,14.1,20.5,...,1849,70.0,7.2,64.6,7.1,7.4,4.6,6044,1328,88.4,5.2,88.4,5.2,0.0,3.1
3,0500000US01125,"Tuscaloosa County, Alabama",170265,923,60.7,2.3,58.1,2.4,4.1,1.2,19694,2008,27.5,6.4,24.7,...,2951,78.4,4.0,77.3,4.0,1.1,1.1,31079,3427,86.7,3.3,85.6,3.4,0.9,0.7
4,0500000US04021,"Pinal County, Arizona",356563,1561,49.3,1.4,45.7,1.5,7.2,1.3,20770,1723,43.4,6.8,38.2,...,4467,66.9,3.2,64.2,3.3,4.0,1.6,38076,3814,78.2,3.8,75.9,4.0,3.0,1.9


In [30]:
unemp = unemp[['NAME', 'S2301_C04_021E']]
unemp.columns = ['countystate', 'unemployment_rate']

In [31]:
unemp.drop([0], inplace = True)
unemp.reset_index(drop = True, inplace = True)

In [32]:
unemp['countystate'] = unemp['countystate'].apply(clean_county)

In [33]:
unemp.head()

,countystate,unemployment_rate
0,"Shelby, Alabama",3.1
1,"Talladega, Alabama",4.1
2,"Tuscaloosa, Alabama",4.0
3,"Pinal, Arizona",6.8
4,"Mendocino, California",6.3


### Merge

In [34]:
gini = pd.merge(gini, unemp, left_on = 'countystate', right_on = 'countystate')

In [35]:
print(inc.shape)
inc = pd.merge(inc, gini, left_on = 'countystate', right_on = 'countystate', how='left')
print(inc.shape)

(18791, 26)
(18852, 28)


In [36]:
# impute missing gini
# todo: use knn with lat/lon

inc['gini'] = pd.to_numeric(inc['gini'])
inc['gini_is_missing'] = inc['gini'].isnull().astype(int)
inc['gini'] = inc['gini'].fillna(inc['gini'].mean())

In [38]:
def clean_n(row):
    if row == 'N':
        return np.nan
    else:
        return row

inc['unemployment_rate'] = inc['unemployment_rate'].apply(clean_n)

In [39]:
inc['unemployment_rate'] = pd.to_numeric(inc['unemployment_rate'])
inc['unemployment_is_missing'] = inc['unemployment_rate'].isnull().astype(int)
inc['unemployment_rate'] = inc['unemployment_rate'].fillna(inc['unemployment_rate'].mean())

In [40]:
inc.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,state,town,poverty,hs_completion,townstate,ZCTA,n_households,med_income,population,density,city,state_id,state_name,lat,lng,crime_rate,students_per_teacher,property_crime,violent_crime,zip,home_price,county_name,countystate,gini,unemployment_rate,gini_is_missing,unemployment_is_missing
0,0,0,0,Alabama,Abbeville,20.7,79.1,"Abbeville, Alabama",36310,2555,40186,6055,12.2,Abbeville,AL,Alabama,31.60296,-85.21630,62.0,13.000000,51.0,11.0,36310,86264,Henry,"Henry, Alabama",0.46135,4.58761,1,1
1,1,1,1,Alabama,Adamsville,16,83.5,"Adamsville, Alabama",35005,2792,50271,7528,89.3,Adamsville,AL,Alabama,33.59515,-87.00089,269.0,17.666667,250.0,19.0,35005,89673,Jefferson,"Jefferson, Alabama",0.49520,6.20000,0,0
2,2,2,2,Alabama,Addison,34.2,85.1,"Addison, Alabama",35540,1075,45074,2436,12.6,Addison,AL,Alabama,34.23388,-87.18817,15.0,15.500000,14.0,1.0,35540,89435,Winston,"Winston, Alabama",0.46135,4.58761,1,1
3,3,3,3,Alabama,Akron,48.7,62.8,"Akron, Alabama",35441,372,36731,1089,6.1,Akron,AL,Alabama,32.85374,-87.73861,NaN,NaN,NaN,NaN,35441,92705,Hale,"Hale, Alabama",0.46135,4.58761,1,1
4,4,4,4,Alabama,Alabaster,11.1,88.8,"Alabaster, Alabama",35007,8628,69691,26328,270.8,Alabaster,AL,Alabama,33.21591,-86.79717,585.0,18.571429,488.0,97.0,35007,167295,Shelby,"Shelby, Alabama",0.45890,3.10000,0,0


In [41]:
inc.shape

(18852, 30)

### Address the nulls

In [43]:
inc.isnull().sum()

Unnamed: 0                     0
Unnamed: 0.1                   0
Unnamed: 0.1.1                 0
state                          0
town                           0
poverty                        0
hs_completion                  0
townstate                      0
ZCTA                           0
n_households                   0
med_income                     0
population                     0
density                        0
city                           0
state_id                       0
state_name                     0
lat                            0
lng                            0
crime_rate                 12614
students_per_teacher        3759
property_crime             12615
violent_crime              12614
zip                            0
home_price                     0
county_name                    0
countystate                    0
gini                           0
unemployment_rate              0
gini_is_missing                0
unemployment_is_missing        0
dtype: int

In [44]:
inc.dropna(inplace = True)

In [45]:
inc.shape

(5827, 30)

In [46]:
# Drop unnecessary cols
inc.drop(
    columns = [
        'Unnamed: 0', 
        'Unnamed: 0.1',
        'Unnamed: 0.1.1',
        'city',
        'state_id'
    ], 
    inplace = True)

In [50]:
inc.head()

,state,town,poverty,hs_completion,townstate,ZCTA,n_households,med_income,population,density,state_name,lat,lng,crime_rate,students_per_teacher,property_crime,violent_crime,zip,home_price,county_name,countystate,gini,unemployment_rate,gini_is_missing,unemployment_is_missing
0,Alabama,Abbeville,20.7,79.1,"Abbeville, Alabama",36310,2555,40186,6055,12.2,Alabama,31.60296,-85.21630,62.0,13.000000,51.0,11.0,36310,86264,Henry,"Henry, Alabama",0.46135,4.58761,1,1
1,Alabama,Adamsville,16.0,83.5,"Adamsville, Alabama",35005,2792,50271,7528,89.3,Alabama,33.59515,-87.00089,269.0,17.666667,250.0,19.0,35005,89673,Jefferson,"Jefferson, Alabama",0.49520,6.20000,0,0
2,Alabama,Addison,34.2,85.1,"Addison, Alabama",35540,1075,45074,2436,12.6,Alabama,34.23388,-87.18817,15.0,15.500000,14.0,1.0,35540,89435,Winston,"Winston, Alabama",0.46135,4.58761,1,1
4,Alabama,Alabaster,11.1,88.8,"Alabaster, Alabama",35007,8628,69691,26328,270.8,Alabama,33.21591,-86.79717,585.0,18.571429,488.0,97.0,35007,167295,Shelby,"Shelby, Alabama",0.45890,3.10000,0,0
5,Alabama,Alabaster,11.1,88.8,"Alabaster, Alabama",35114,3022,80948,8624,223.3,Alabama,33.22306,-86.87245,585.0,18.571429,488.0,97.0,35114,200120,Shelby,"Shelby, Alabama",0.45890,3.10000,0,0


**Fix Types**

In [48]:
inc['poverty'] = inc['poverty'].astype(float)

In [49]:
inc['hs_completion'] = inc['hs_completion'].astype(float)

### Fix 0 population

In [51]:
# 15 rows have 0 for population which causes infinity crime rate per capita
inc['population'].value_counts()

3708    4
6572    4
1464    4
2336    3
4395    3
       ..
5256    1
7305    1
9354    1
5260    1
8192    1
Name: population, Length: 5227, dtype: int64

In [52]:
inc['n_households'].value_counts()

5848     6
0        5
1926     5
1470     5
2902     5
        ..
9214     1
13308    1
11259    1
1018     1
12304    1
Name: n_households, Length: 4470, dtype: int64

In [54]:
# Fill the 0 population towns
inc.loc[inc['population'] == 0, 'population'] = 100 #inc['population'].median()

In [55]:
# Fill the 0 household towns
inc.loc[inc['n_households'] == 0, 'n_households'] = 10

In [56]:
# Target variable: crime rate per capita
#inc['crime_rate_pc'] = inc['crime_rate'] / inc['n_households']
inc['crime_rate_pc'] = inc['crime_rate'] / inc['population']

### Grouping by town

- dataframe is by zip so towns with multiple zips are counted multiple times
- should be grouped by town

- These should be averaged
 - med_income
 - hs_completion
 - poverty
 - density

In [57]:
# dataframe is by zip so towns with multiple zips are counted multiple times
# to rectify, we will groupby townstate, then re add the state col
inc['townstate'].value_counts()

Naples, Florida            14
Vero Beach, Florida         6
Johnstown, Pennsylvania     6
Saginaw, Michigan           6
Tyler, Texas                6
                           ..
Fort Jones, California      1
Mound City, Missouri        1
Reed City, Michigan         1
American Falls, Idaho       1
Rifle, Colorado             1
Name: townstate, Length: 5371, dtype: int64

In [58]:
testinc = inc.groupby('townstate', as_index=False).agg(
    {
        "poverty" : np.mean,
        "med_income" : np.mean,
        "hs_completion" : np.mean,
        "n_households" : np.sum,
        "population" : np.sum,
        "density" : np.mean,
        "crime_rate" : np.mean,
        "property_crime" : np.mean,  # already at town-level
        "violent_crime" : np.mean,
        "students_per_teacher" : np.mean,
        "gini" : np.mean,
        "lat" : np.mean,
        "lng" : np.mean,
        "unemployment_rate" : np.mean,
        "home_price" : np.mean
    }
)

In [59]:
testinc['crime_rate_pc'] = testinc['crime_rate'] / testinc['population']

In [60]:
testinc['vcrime_rate_pc'] = testinc['violent_crime'] / testinc['population']
testinc['pcrime_rate_pc'] = testinc['property_crime'] / testinc['population']

In [61]:
testinc['people_per_household'] = testinc['population'] / testinc['n_households']

In [62]:
testinc.shape

(5371, 20)

In [63]:
testinc['state'] = testinc['townstate'].apply(lambda x: x.split(', ')[1])

In [22]:
# Drop outliers
#testinc = inc.loc[(inc['crime_rate_pc'] > 0) & (inc['crime_rate_pc'] < 1)]

In [65]:
testinc.head()

,townstate,poverty,med_income,hs_completion,n_households,population,density,crime_rate,property_crime,violent_crime,students_per_teacher,gini,lat,lng,unemployment_rate,home_price,crime_rate_pc,vcrime_rate_pc,pcrime_rate_pc,people_per_household,state
0,"Abbeville, Alabama",20.7,40186.0,79.1,2555,6055,12.2,62.0,51.0,11.0,13.000000,0.46135,31.60296,-85.21630,4.58761,86264.0,0.010239,0.001817,0.008423,2.369863,Alabama
1,"Abbeville, Louisiana",31.5,47483.0,70.7,9292,25897,42.4,591.0,474.0,117.0,14.875000,0.46135,29.89813,-92.19967,4.58761,129370.0,0.022821,0.004518,0.018303,2.787021,Louisiana
2,"Abbeville, South Carolina",39.9,34292.0,75.3,4925,13006,19.9,229.0,199.0,30.0,14.600000,0.46135,34.17875,-82.42871,4.58761,128638.0,0.017607,0.002307,0.015301,2.640812,South Carolina
3,"Aberdeen, Idaho",13.8,43793.0,61.3,1127,3533,5.0,22.0,21.0,1.0,20.000000,0.46135,43.01939,-112.86185,4.58761,128794.0,0.006227,0.000283,0.005944,3.134871,Idaho
4,"Aberdeen, Maryland",17.2,60489.0,88.0,9538,24453,277.6,330.0,251.0,79.0,12.285714,0.41070,39.51075,-76.20061,3.70000,213991.0,0.013495,0.003231,0.010265,2.563745,Maryland


In [66]:
testinc.shape

(5371, 21)

In [67]:
testinc.to_csv('../data/testinc.csv')